In [56]:
import dash
from dash import Input, Output, html, dcc, State, dash_table
import dash_bootstrap_components as dbc
import math
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from cs50 import SQL
import time
from datetime import date, datetime, timedelta
from pandas.tseries.offsets import DateOffset
import plotly.graph_objects as go
import numpy as np

In [57]:
db = SQL("sqlite:///project.db")

In [58]:
query = db.execute("SELECT projects.name project, task_id, tasks.name task, demand, people_id, people.name person, teams.name team, start_date, end_date, hours_day, total_hours FROM TaskAssign, Tasks, People, Projects, Teams WHERE task_id = tasks.id AND people_id = people.id AND tasks.project_id = projects.id AND people.team_id = teams.id")
df = pd.DataFrame(query)

df

,project,task_id,task,demand,people_id,person,team,start_date,end_date,hours_day,total_hours
0,CE Serra do Tigre,1,Relatório de Edificações,32,2,Max,Socio,2023-03-29,2023-03-30,8,16
1,CE Serra do Tigre,1,Relatório de Edificações,32,7,Raquel,Socio,2023-03-29,2023-03-30,8,16
2,CE Cajuina A,2,Diagnóstico PMIS,24,5,Mariana,Socio,2023-03-29,2023-03-31,8,24
3,UFV Oliveira dos Brejinhos,3,Mobilização de Campo,4,7,Raquel,Socio,2023-03-31,2023-03-31,4,4
4,CE Babilônia Centro,4,Programas PBA LT,16,6,Edimara,Socio,2023-03-30,2023-03-31,8,16
5,CE Rio do Vento Expansão,5,Diagnóstico PMIS,32,5,Mariana,Socio,2023-04-03,2023-04-06,8,32
6,CE Queimada Nova IV,6,Diagnóstico de ADA,8,6,Edimara,Socio,2023-03-29,2023-03-29,8,8
7,CE Queimada Nova IV,7,Análise Integrada,4,6,Edimara,Socio,2023-03-30,2023-03-30,4,4
8,CE Queimada Nova IV,8,Revisão AIA/RIMA/Programas,8,6,Edimara,Socio,2023-03-30,2023-03-31,4,8
9,CE Cajuina 5L,9,Revisão Inventário Turístico,4,7,Raquel,Socio,2023-04-03,2023-04-03,4,4


In [79]:
report1 = df.loc[:,['project','task', 'demand', 'total_hours']].groupby(['project', 'task']).agg({'demand': 'first', 'total_hours':'sum'}).sort_values(by=['project']).reset_index()

report1['% Assigned'] = report1['total_hours']/report1['demand']*100

report1 = report1.rename(columns={'project':'Project','task':'Task','demand':'Demand', 'total_hours':'Assigned(Hours)'}, errors='raise')

report1

,Project,Task,Demand,Assigned(Hours),% Assigned
0,CE Babilônia Centro,Campo - Mapeamento de Stakeholders,40,40,100.0
1,CE Babilônia Centro,Diagnóstico de Mapeamento,128,128,100.0
2,CE Babilônia Centro,Itemização de Relatório,6,6,100.0
3,CE Babilônia Centro,Programas PBA LT,16,16,100.0
4,CE Babilônia Centro,Reuniões Comunitárias,24,24,100.0
5,CE Babilônia Centro,Revisão Final do Mapeamento,16,16,100.0
6,CE Cajuina 5L,Revisão Inventário Turístico,4,4,100.0
7,CE Cajuina A,Diagnóstico PMIS,24,24,100.0
8,CE Queimada Nova IV,Diagnóstico de ADA,8,8,100.0
9,CE Queimada Nova IV,Revisão AIA/RIMA/Programas,8,8,100.0


In [80]:
query = db.execute("SELECT projects.name Project, tasks.name Task, demand Demand FROM Tasks, Projects WHERE tasks.project_id = projects.id")
tasks = pd.DataFrame(query)

result = pd.merge(report1, tasks, on="")





,Project,Task,Demand
0,CE Serra do Tigre,Relatório de Edificações,32
1,CE Cajuina A,Diagnóstico PMIS,24
2,UFV Oliveira dos Brejinhos,Mobilização de Campo,4
3,CE Babilônia Centro,Programas PBA LT,16
4,CE Rio do Vento Expansão,Diagnóstico PMIS,32
5,CE Queimada Nova IV,Diagnóstico de ADA,8
6,CE Queimada Nova IV,Análise Integrada,4
7,CE Queimada Nova IV,Revisão AIA/RIMA/Programas,8
8,CE Cajuina 5L,Revisão Inventário Turístico,4
9,CE Cajuina 5L,Revisão Itens,8


In [61]:
fig = go.Figure(data=[go.Table(
    header=dict(values=['Project', 'Task', 'Start Date', 'End Date', 'Demand (Hours)', 'Assigned (Hours)', '% Assigned']),
    cells=dict(values=report1.transpose().values.tolist()))
])

fig.update_layout(
        template='plotly_dark',
    )

fig.show()

In [62]:
fig = px.timeline(df, x_start="start_date", x_end="end_date", y="task", color='team')
fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up
fig.layout.template = 'plotly_dark'
fig.show()

In [63]:
dates = pd.bdate_range(df.start_date.min(), df.end_date.max())

demand = pd.DataFrame(
    columns = df.team.unique(),
    index = dates).fillna(0)

demand = demand.rename_axis("team", axis="columns")

capacity = pd.DataFrame(
    columns = df.team.unique(),
    index = dates).fillna(0)

capacity = capacity.rename_axis('team', axis='columns')

for index, row in df.iterrows():
    demand.loc[row['start_date']:row['end_date'], row.team] += row.hours_day;
    
demand['Assigned'] = 0
demand['Assigned'] = demand.sum(axis=1)
    
people = pd.DataFrame(db.execute('SELECT teams.name team, people.name FROM people, teams WHERE team_id = teams.id'))

for team in df.team.unique():
    capacity[team] = people[people.team == team]['team'].count()*8

capacity['Total'] = 0
capacity['Total'] = capacity.sum(axis=1)
    
    

fig = px.bar(
    demand['Assigned'], 
    barmode='overlay', 
    color_discrete_sequence=px.colors.qualitative.Dark24, 
    opacity=1,
    labels={
        "index": "",
        "value": "Hours",
        "variable":""

    },
    title="Hours Assigned",
)
fig.add_trace(go.Bar(x=capacity.index , y=capacity['Total'], opacity=0.1, name='Capacity'))
fig.layout.template = 'plotly_dark'




In [64]:
demand

team,Socio,Assigned
2023-03-29,40,40
2023-03-30,48,48
2023-03-31,32,32
2023-04-03,28,28
2023-04-04,32,32
2023-04-05,22,22
2023-04-06,24,24
2023-04-07,0,0
2023-04-10,20,20
2023-04-11,24,24


In [65]:
db.execute("SELECT SUM(total_hours) hours FROM TaskAssign WHERE task_id = 10")[0]['hours']

In [66]:
demand = demand.drop('Assigned', axis=1)
fig = px.bar(demand, facet_col='team', facet_col_wrap=2,
             labels={
                     "index": "",
                     "value": "",
                     "team": "Team"
                 },
            title="Hours Assigned",
            color_discrete_sequence=px.colors.qualitative.Alphabet)
fig.layout.template = 'plotly_dark'
fig.show()

In [67]:
start=df.start_date.min()
end =df.end_date.max()

fig1 = go.Figure()

fig1.add_trace(go.Indicator(
    mode = "number",
    value = demand.loc[start:end, 'Assigned'].sum(),
    title= {'text': "Hours Assigned"},
    domain = {'row': 0, 'column': 0},
    #delta = {'reference': 400, 'relative': True, 'position' : "top"}
))

fig1.add_trace(go.Indicator(
    mode = "number+delta",
    value = capacity.loc[start:end, 'Total'].sum(),
    title= {'text': "Capacity"},
    delta = {'reference': demand.loc[start:end, 'Assigned'].sum()},
    domain = {'row': 0, 'column': 1},
))

fig1.update_layout(
    grid = {'rows': 1, 'columns': 2, 'pattern': "independent",},
    template='plotly_dark'
)

fig.show()

KeyError: 'Assigned'

In [68]:
import plotly.io as pio
pio.templates

Templates configuration
-----------------------
    Default template: 'plotly'
    Available templates:
        ['ggplot2', 'seaborn', 'simple_white', 'plotly',
         'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
         'ygridoff', 'gridon', 'none']

In [69]:
df['end_date'] = pd.to_datetime(df['end_date'])
df['start_date'] = pd.to_datetime(df['start_date'])
df['end_date'] = df['end_date'] + pd.to_timedelta(23.99, unit='h')

In [70]:
df['end_date'] = df['end_date'] + pd.to_timedelta(23.99, unit='h')

In [71]:
df

,project,task_id,task,demand,people_id,person,team,start_date,end_date,hours_day,total_hours
0,CE Serra do Tigre,1,Relatório de Edificações,32,2,Max,Socio,2023-03-29,2023-03-30 23:59:24,8,16
1,CE Serra do Tigre,1,Relatório de Edificações,32,7,Raquel,Socio,2023-03-29,2023-03-30 23:59:24,8,16
2,CE Cajuina A,2,Diagnóstico PMIS,24,5,Mariana,Socio,2023-03-29,2023-03-31 23:59:24,8,24
3,UFV Oliveira dos Brejinhos,3,Mobilização de Campo,4,7,Raquel,Socio,2023-03-31,2023-03-31 23:59:24,4,4
4,CE Babilônia Centro,4,Programas PBA LT,16,6,Edimara,Socio,2023-03-30,2023-03-31 23:59:24,8,16
5,CE Rio do Vento Expansão,5,Diagnóstico PMIS,32,5,Mariana,Socio,2023-04-03,2023-04-06 23:59:24,8,32
6,CE Queimada Nova IV,6,Diagnóstico de ADA,8,6,Edimara,Socio,2023-03-29,2023-03-29 23:59:24,8,8
7,CE Queimada Nova IV,7,Análise Integrada,4,6,Edimara,Socio,2023-03-30,2023-03-30 23:59:24,4,4
8,CE Queimada Nova IV,8,Revisão AIA/RIMA/Programas,8,6,Edimara,Socio,2023-03-30,2023-03-31 23:59:24,4,8
9,CE Cajuina 5L,9,Revisão Inventário Turístico,4,7,Raquel,Socio,2023-04-03,2023-04-03 23:59:24,4,4


In [72]:
fig6 = px.timeline(df, 
                       x_start='start_date', 
                       x_end='end_date', 
                       y='person', 
                       color='person',
                       labels={'person': ''},
                       title='People Timeline',)
    
fig6.update_yaxes(autorange='reversed') 

fig6.update_layout(
    template='plotly_dark',
    legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.4,
            xanchor="center",
            x=0.5))

fig6